<a href="https://colab.research.google.com/github/Nalzmami/Hayyan/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install whisper
!pip install gradio
!pip install openai
!pip install --upgrade openai
!pip install git+https://github.com/openai/whisper.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41121 sha256=cdbdc1093260afd20a94a46ffab0ae29367b2859d7ea4d8ec5dfcc3a7f0b0373
  Stored in directory: /root/.cache/pip/wheels/aa/7c/1d/015619716e2facae6631312503baf3c3220e6a9a3508cb14b6
Successfully built whisper
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 

In [ ]:
import whisper
import numpy as np
from openai import OpenAI
import gradio as gr
import soundfile as sf
import os
import tempfile

In [ ]:
client = OpenAI(api_key='')

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")


In [ ]:
def speech_to_txt(input):
    model = whisper.load_model("base.en")
    print(model.device)

    # Transcribe the audio
    result = model.transcribe(input)
    return result["text"]


#microsoft Translation API
import requests, uuid, json

def response_to_text(response):
    try:
        text = response[0]['translations'][0]["text"]
    except (KeyError, IndexError):
        text = "Translation error occurred"
    return text


def translate_text_API(input: str):

    key = ""
    endpoint = "https://api.cognitive.microsofttranslator.com/"
    location = "uaenorth"
    path = '/translate'
    constructed_url = endpoint + path

    params = {
        'api-version': '3.0',
        'from': 'en',
        'to': ['ar']
    }

    headers = {
        'Ocp-Apim-Subscription-Key': key,
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    body = [{
        'text': input
    }]

    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    output = response_to_text(response)  # You can include this line if needed
    return output



import gradio as gr
import openai


openai.api_key = ''

def get_response(thread):
    response = client.beta.threads.messages.list(thread_id=thread.id, order="asc")
    return [msg for msg in response.data]

def generate_Notes(whisperText, prompt=None):
    if prompt is None:
        prompt = f'''eyour role is to note taking,
        take notes of the following lecture text and show the most important titles and underneath it write notes follow these instructions:
        1-write the titles in numerical order and underneath them write the notes, don't add information that is not in the lecture text
        2-be organized and keep the notes very short
        3-if you can, give examples from the same lecture text underneath the notes
        4-be organized and importantly leave space between each title point, in the end just show the notes:\n\n{whisperText}'''
    else:
        prompt = f'"{whisperText}"\n{prompt}'

    assistant_id = ""

    thread = client.beta.threads.create()
    submit_message(assistant_id, thread, prompt)

    messages = get_response(thread)
    assistant_messages = [msg.content for msg in messages if msg.role == 'assistant']

    if assistant_messages:
        notes = assistant_messages[-1]  # Get the last message
    else:
        notes = "No notes"  # Handle the case where no response is received

    return notes

In [ ]:
def generate(input):
    if isinstance(input, tuple):
        sample_rate, audio_data = input
        with tempfile.NamedTemporaryFile(delete=False, suffix='.wav') as tmp_file:
            sf.write(tmp_file, audio_data, sample_rate)
            file_path = tmp_file.name
            output = speech_to_txt(file_path)
            os.remove(file_path)  # Clean up the temp file
    else:
        # If the input is a file path
        output = speech_to_txt(input)

    output_2 = generate_Notes(output)
    output__2 = translate_text_API(output_2)
    return output_2, output__2

In [ ]:
blocks = gr.Blocks()
with blocks as demo:
    drop = gr.Audio(label="Drop your audio file here!")

    with gr.Row():
        btn1 = gr.Button("Use Uploaded Audio")

    with gr.Row():
        output__2 = gr.Textbox(label="Notes")
        output_translation = gr.Textbox(label="Arabic notes")

    # Attach the click event outside the 'with gr.Row():' block
    btn1.click(fn=generate, inputs=drop, outputs=[output__2, output_translation])

demo.queue().launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://1ad501a1675764e7e5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
